In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

%matplotlib inline

In [11]:
url = 'https://raw.githubusercontent.com/ngophuong/VNStock/main/excel_mbb.csv'
df = pd.read_csv(url, index_col = 0)
df['datetime']=pd.to_datetime(df['<DTYYYYMMDD>'],format='%Y%m%d')
df['Week_no']= df['datetime'].dt.isocalendar().week
df['Year_no']= df['datetime'].dt.year
df['Period'] = df['Year_no'].astype(str) + str("-") + df['Week_no'].astype(str)
stock = 'MBB'
df.head(3)


,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,datetime,Week_no,Year_no,Period
<Ticker>,,,,,,,,,,
MBB,20111101,3.4400,3.6145,3.4400,3.4400,2719290,2011-11-01,44,2011,2011-44
MBB,20111102,3.4400,3.4400,3.2904,3.2904,1255220,2011-11-02,44,2011,2011-44
MBB,20111103,3.2904,3.2904,3.1408,3.2655,1193910,2011-11-03,44,2011,2011-44


In [12]:
df = df.rename(columns={'<CloseFixed>':'<Close>'})
df = df[['<Close>','datetime','<Volume>','Period','Year_no']]

In [13]:
#plot linechart for stock price
short_rolling = df['<Close>'].rolling(window=20).mean()

ema_long = df['<Close>'].ewm(span=50, adjust=False).mean()
ema_short = df['<Close>'].ewm(span=20, adjust=False).mean()
line_price = df['<Close>']
fig = go.Figure()
#px.line(df_hpg, x='datetime', y='<Close>', labels={'x':'Date', 'y':'Price'})
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=line_price, 
                        mode='lines', name=stock))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_short, 
                        mode='lines', name='Span 20-days EMA'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_long, 
                        mode='lines', name='Span 50-days EMA'))


In [17]:
#px.violin(df_tips, y="tip", x="smoker", color="sex", box=True, points="all",
#          hover_data=df_tips.columns)

fig = go.Figure()
px.violin(df,x='Year_no',y='<Close>',box =True, points='all',
         hover_data=df.columns, title = 'STOCK Violin chart')

In [18]:
df.tail(3)

,<Close>,datetime,<Volume>,Period,Year_no
<Ticker>,,,,,
MBB,28.40,2021-12-08,7357500,2021-49,2021
MBB,28.55,2021-12-09,4588800,2021-49,2021
MBB,28.35,2021-12-10,6326300,2021-49,2021


In [19]:
#HDFC_df[‘Day_Perc_Change’] = HDFC_df[‘Adj Close’].pct_change()*100
df['Daily_return']=df['<Close>'].pct_change()*100
df.dropna(axis = 0, inplace = True)
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return
<Ticker>,,,,,,
MBB,3.2904,2011-11-02,1255220,2011-44,2011,-4.348837
MBB,3.2655,2011-11-03,1193910,2011-44,2011,-0.756747
MBB,3.2156,2011-11-04,483860,2011-44,2011,-1.528097


In [20]:
fig = go.Figure()
px.line(df, x='datetime', y='Daily_return', labels={'x':'Date', 'y':'Daily_Return_%'})

In [ ]:
df['Daily_return'].describe()

count    1499.000000
mean       -0.035633
std         1.913147
min        -6.472492
25%        -0.905387
50%         0.000000
75%         0.724638
max        12.146893
Name: Daily_return, dtype: float64

In [ ]:
fig = go.Figure()
px.histogram(df, x="Daily_return",histnorm='percent')
#ff.create_distplot(df, group_labels)

In [ ]:
df['<Close>'].describe()

count    1499.000000
mean       19.932722
std         5.202727
min        12.900000
25%        15.100000
50%        20.400000
75%        22.750000
max        36.800000
Name: <Close>, dtype: float64

In [ ]:
dfig = go.Figure()
px.histogram(df.head(60), x="Daily_return",histnorm='percent')

In [ ]:


df['Volumn'] = df['<Volume>']/1000000
Volumnshort_rolling = df['Volumn'].rolling(window=7).mean()

fig = go.Figure()
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=df['Daily_return'], 
                        mode='lines', name='Dailly_Retun'))
fig.add_trace(go.Scatter(x=df.datetime, y=df['Volumn'], 
                        mode='lines+markers', name='Volumn'))

fig.add_trace(go.Scatter(x=df.datetime, y=Volumnshort_rolling, 
                      mode ='lines', name='Span_7_Rolling_vol'))

fig.add_trace(go.Scatter(x=df.datetime, y=df['<Close>'], 
                        mode='lines+markers', name='Price'))

In [ ]:
df['Volumn'].head(30).describe()

count    30.000000
mean     16.553037
std       5.022009
min       9.917000
25%      13.208350
50%      15.367000
75%      17.844875
max      33.236600
Name: Volumn, dtype: float64

In [ ]:
df['Volumn'].head(10).describe()

count    10.000000
mean     15.545230
std       3.543777
min      10.054300
25%      13.423925
50%      14.675150
75%      17.458750
max      22.558600
Name: Volumn, dtype: float64

In [ ]:
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return,Volumn
<Ticker>,,,,,,,
MBB,29.6,2021-04-27,10054300,2021-17,2021,-2.148760,10.0543
MBB,29.6,2021-04-26,14253600,2021-17,2021,0.000000,14.2536
MBB,30.3,2021-04-23,16710100,2021-16,2021,2.364865,16.7101
